In [22]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential

In [23]:
df = pd.read_csv('processedData.csv')

/home/pyf/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
def train_test(df, norm = False):
    X = df.drop(['Elapsed time', 'Driver', 'marker', 'stress'], axis = 1)
    y = df['stress']
    if norm:
        X=(X-X.min())/(X.max()-X.min()) # Normalising to rescale all the features. 
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_Y = encoder.transform(y)
    trainX, testX, trainy, testy = train_test_split(X, encoded_Y, test_size = 0.2 )
    num_features = len(X.columns)
    return trainX, testX, trainy, testy, num_features

def make_model(input_shape):
    '''
    This architecture is the same architecture as proposed by Azar et al.
    '''
    model = Sequential()
    model.add(Dense(60, input_dim=input_shape, activation='relu'))
    model.add(Dense(60, activation = 'relu'))
    model.add(Dense(60, activation = 'relu'))
    model.add(Dense(60, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])
    model.summary()
    return model

## Training model without Encoding:

In [27]:
trainX, testX, trainy, testy, num_features = train_test(df)
model = make_model(input_shape = num_features)
model.fit(trainX, trainy, epochs = 125,shuffle=True ,validation_split = 0.2)
op = model.evaluate(testX, testy)
print("Test Accuracy=", op[1]*100, '%')
print("Test Loss=", op[0])
model.save('NN_v1.h5')

## Encoding data using Autoencoders

Training autoencoder:

In [55]:
encoding_dim = 5
input_dim = trainX.shape[1]

input_vec = Input(shape=(input_dim,))
encoded = Dense(32, activation='relu')(input_vec)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)


decoded = Dense(8, activation='sigmoid')(encoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(32, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)


# encoder = Model(input_vec, encoded)
# encoded_input = Input(shape=(encoding_dim,))
# decoder_layer = autoencoder.layers[5]
# decoder = Model(encoded_input, decoder_layer(encoded_input))
# decoder.summary()

autoencoder = Model(input_vec, decoded)
autoencoder.summary()

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(trainX, trainX,
                epochs=100,
                batch_size=256,
                shuffle=True)

ValueError: Output tensors to a Model must be the output of a Keras `Layer` (thus holding past layer metadata). Found: <keras.layers.core.Dense object at 0x7fc190554f98>

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 26)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 32)                864       
_________________________________________________________________
dense_55 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_57 (Dense)             (None, 5)                 45        
_________________________________________________________________
dense_58 (Dense)             (None, 8)                 48        
_________________________________________________________________
dense_59 (Dense)             (None, 16)                144

Encoding and decoding training data:

In [58]:
trainX_anc = autoencoder.predict(trainX) # Encoding and decoding the training data
testX_anc = autoencoder.predict(testX) # Encoding and decoding the testing data
print(trainX_anc.shape)
print(testX_anc.shape)

(38436, 26)
(9609, 26)


**Training NN model on encoded-decoded data**

In [59]:
postEnc_model = make_model(input_shape = trainX_anc.shape[1])
postEnc_model.fit(trainX_anc, trainy, epochs = 125,shuffle=True ,validation_split = 0.2)
op = postEnc_model.evaluate(testX_anc, testy)
print("Test Accuracy after encoding-decoding =", op[1]*100, '%')
print("Test Loss=", op[0])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 60)                1620      
_________________________________________________________________
dense_124 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_125 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_126 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_127 (Dense)            (None, 1)                 61        
Total params: 12,661
Trainable params: 12,661
Non-trainable params: 0
_________________________________________________________________
Train on 30748 samples, validate on 7688 samples
Epoch 1/125
30748/30748 [==============================] - 4s 115us

30748/30748 [==============================] - 3s 100us/step - loss: 0.4993 - accuracy: 0.7506 - val_loss: 0.4926 - val_accuracy: 0.7534
Epoch 49/125
30748/30748 [==============================] - 3s 99us/step - loss: 0.4995 - accuracy: 0.7500 - val_loss: 0.5050 - val_accuracy: 0.7518
Epoch 50/125
30748/30748 [==============================] - 3s 99us/step - loss: 0.5006 - accuracy: 0.7492 - val_loss: 0.4982 - val_accuracy: 0.7540
Epoch 51/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4982 - accuracy: 0.7511 - val_loss: 0.4915 - val_accuracy: 0.7575
Epoch 52/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.5004 - accuracy: 0.7481 - val_loss: 0.5020 - val_accuracy: 0.7553
Epoch 53/125
30748/30748 [==============================] - 3s 98us/step - loss: 0.4995 - accuracy: 0.7498 - val_loss: 0.4927 - val_accuracy: 0.7560
Epoch 54/125
30748/30748 [==============================] - 3s 98us/step - loss: 0.5018 - accuracy: 0.7524 - val_los

Epoch 103/125
30748/30748 [==============================] - 3s 104us/step - loss: 0.4898 - accuracy: 0.7539 - val_loss: 0.4860 - val_accuracy: 0.7605
Epoch 104/125
30748/30748 [==============================] - 3s 103us/step - loss: 0.4904 - accuracy: 0.7538 - val_loss: 0.4896 - val_accuracy: 0.7516
Epoch 105/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4891 - accuracy: 0.7551 - val_loss: 0.4864 - val_accuracy: 0.7653
Epoch 106/125
30748/30748 [==============================] - 3s 102us/step - loss: 0.4905 - accuracy: 0.7546 - val_loss: 0.4913 - val_accuracy: 0.7581
Epoch 107/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4911 - accuracy: 0.7542 - val_loss: 0.4872 - val_accuracy: 0.7572
Epoch 108/125
30748/30748 [==============================] - 3s 101us/step - loss: 0.4894 - accuracy: 0.7534 - val_loss: 0.5024 - val_accuracy: 0.7530
Epoch 109/125
30748/30748 [==============================] - 3s 100us/step - loss: 0.4899 - ac

**Accuracy went down to 75% (from 95% acc without encoding data) after encoding-decoding the data**

### Autoencoder v2

In [77]:
encoding_dim = 5
input_dim = trainX.shape[1]

input_vec = Input(shape=(input_dim,))
encoded = Dense(16, activation='relu')(input_vec)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

decoded = Dense(8, activation='sigmoid')(encoded)
decoded = Dense(8, activation='sigmoid')(decoded)
decoded = Dense(8, activation='sigmoid')(decoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(16, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder2 = Model(input_vec, decoded)
autoencoder2.summary()

autoencoder2.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder2.fit(trainX, trainX,
                epochs=100,
                batch_size=256,
                shuffle=True)

Model: "model_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 26)                0         
_________________________________________________________________
dense_208 (Dense)            (None, 16)                432       
_________________________________________________________________
dense_209 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_210 (Dense)            (None, 16)                272       
_________________________________________________________________
dense_211 (Dense)            (None, 8)                 136       
_________________________________________________________________
dense_212 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_213 (Dense)            (None, 8)                 72 

38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 68/100
38436/38436 [==============================] - 1s 22us/step - loss: 0.4579
Epoch 69/100
38436/38436 [==============================] - 1s 22us/step - loss: 0.4579
Epoch 70/100
38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 71/100
38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 72/100
38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 73/100
38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 74/100
38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 75/100
38436/38436 [==============================] - 1s 21us/step - loss: 0.4579
Epoch 76/100
38436/38436 [==============================] - 1s 22us/step - loss: 0.4579
Epoch 77/100
38436/38436 [==============================] - 1s 22us/step - loss: 0.4579
Epoch 78/100
38436/38436 [===================

In [75]:
trainX_anc = autoencoder2.predict(trainX) # Encoding and decoding the training data
testX_anc = autoencoder2.predict(testX) # Encoding and decoding the testing data
print(trainX_anc.shape)
print(testX_anc.shape)

model_enc2 = make_model(input_shape = trainX_anc.shape[1]) 
model_enc2.fit(trainX_anc, trainy, epochs = 100,shuffle=True ,validation_split = 0.2)
op = model_enc2.evaluate(testX_anc, testy)
print("Test Accuracy after encoding-decoding =", op[1]*100, '%')
print("Test Loss=", op[0])

(38436, 26)
(9609, 26)
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_189 (Dense)            (None, 60)                1620      
_________________________________________________________________
dense_190 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_191 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_192 (Dense)            (None, 60)                3660      
_________________________________________________________________
dense_193 (Dense)            (None, 1)                 61        
Total params: 12,661
Trainable params: 12,661
Non-trainable params: 0
_________________________________________________________________
Train on 30748 samples, validate on 7688 samples
Epoch 1/100
30748/30748 [===================

KeyboardInterrupt: 

**